# The CHAIN!

In [3]:
from dotenv import load_dotenv
import os

import pandas as pd

import openai

In [4]:
_ = load_dotenv("config.env")
openai.api_key = os.environ['OPENAI_API_KEY']

model_name = "gpt-3.5-turbo-0301"

In [5]:
df = pd.read_csv('data/amazonreviews.tsv', sep='\t')
df.head()

,label,review
0,pos,Stuning even for the non-gamer: This sound tra...
1,pos,The best soundtrack ever to anything.: I'm rea...
2,pos,Amazing!: This soundtrack is my favorite music...
3,pos,Excellent Soundtrack: I truly like this soundt...
4,pos,"Remember, Pull Your Jaw Off The Floor After He..."


## Chains

A chain is the combination of an LLM and a prompt.

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

## Basic Chain

In [19]:
# Load the LLM
llm = ChatOpenAI(temperature=0.9, model=model_name)

# Create a prompt template
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)
print(prompt.input_variables)

# Create a chain
chain = LLMChain(llm=llm, prompt=prompt)

['product']


In [20]:
product = "Queen Size Sheet Set"

chain.run(product)

'Royal Rest Bedding Co.'

## Sequential chains

Runs a sequence of chains one after another.

There are two types of Sequential chains: The simple sequential chain and the sequential chain.

The "simple sequential chain" is a straight forward chain that runs the chains in the order they are defined.

![Simple Sequential Chain](imgs/simple-sequential-chain.png)

The sequential chain allows for more complex chaining.

![Sequential Chain](imgs/sequential-chain.png)

### Simple sequential chain

In [24]:
from langchain.chains import SimpleSequentialChain

In [25]:
llm = ChatOpenAI(temperature=0.9, model=model_name)

# Chain 1
prompt1 = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)
chain1 = LLMChain(llm=llm, prompt=prompt1)

# Chain 2
prompt2 = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
chain2 = LLMChain(llm=llm, prompt=prompt2)

# Combine the chains
chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

In [26]:
chain.run(product)



> Entering new SimpleSequentialChain chain...
Royal Linens
Royal Linens is a luxury bedding and bath linens company, offering high-quality products made from the finest materials.

> Finished chain.


'Royal Linens is a luxury bedding and bath linens company, offering high-quality products made from the finest materials.'

### Sequential chain

In [1]:
from langchain.chains import SequentialChain

In [21]:
llm = ChatOpenAI(temperature=0.9, model=model_name)

# Chain 1: Translate to english
# takes in the `Review` and outputs `English_Review`
prompt1 = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
chain1 = LLMChain(
    llm=llm, prompt=prompt1, output_key="English_Review"
)

# Chain 2: Summarize
# takes in `English_Review` and outputs `summary`
prompt2 = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
chain2 = LLMChain(
    llm=llm, prompt=prompt2, output_key="summary"
)

# Chain 3: Translate original review to english
# takes in `Review` and outputs `language`
prompt3 = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
chain3 = LLMChain(
    llm=llm, prompt=prompt3, output_key="language"
)

# Chain 4: follow up message
# takes in `summary`, `language` and outputs `followup_message`
prompt4 = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
chain4 = LLMChain(
    llm=llm, prompt=prompt4, output_key="followup_message"
)

# Combining the chains
# takes in `Review` and outputs `English_Review`, `summary`, `followup_message`
chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [19]:
review = """Un chef-d'œuvre absolu : je suis très sûr \
que vous qui prenez le temps de lire ceci avez joué au \
jeu au moins une fois et avez entendu au moins quelques-unes \
des pistes ici. Et que vous en soyez conscient ou non, la \ 
musique de Mitsuda a contribué grandement à l'ambiance \
de chaque minute du jeu entier. Composée de 3 CD et de \
plusieurs chansons (je n'ai pas de compte exact), toutes \
émouvantes et remarquables, cette bande originale est \
celle que je vous assure que vous n'oublierez pas. Elle \
a tout pour chaque auditeur, des morceaux rapides et \
énergiques (Dancing the Tokage ou Termina Home) aux plus \
lents et plus hantés (Dragon God), en passant par une \
composition magnifiquement réalisée (Time's Scar), jusqu'à \
même quelques chansons fantastiques (Radical Dreamers). \
C'est l'une des meilleures bandes originales de jeux vidéo \
disponibles et sûrement la meilleure de Mitsuda. ^_^
"""

chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Un chef-d'œuvre absolu : je suis très sûr que vous qui prenez le temps de lire ceci avez joué au jeu au moins une fois et avez entendu au moins quelques-unes des pistes ici. Et que vous en soyez conscient ou non, la \\ \nmusique de Mitsuda a contribué grandement à l'ambiance de chaque minute du jeu entier. Composée de 3 CD et de plusieurs chansons (je n'ai pas de compte exact), toutes émouvantes et remarquables, cette bande originale est celle que je vous assure que vous n'oublierez pas. Elle a tout pour chaque auditeur, des morceaux rapides et énergiques (Dancing the Tokage ou Termina Home) aux plus lents et plus hantés (Dragon God), en passant par une composition magnifiquement réalisée (Time's Scar), jusqu'à même quelques chansons fantastiques (Radical Dreamers). C'est l'une des meilleures bandes originales de jeux vidéo disponibles et sûrement la meilleure de Mitsuda. ^_^\n",
 'English_Review': 'An absolute masterpiece: I am very sure that those of you who are taking th

## Router chain

![Router Chain](imgs/router-chain.png)

In [25]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

**1. Define the prompts**

In [22]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [24]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

**2. Define the chains**

In [26]:
llm = ChatOpenAI(temperature=0, model=model_name)

destination_chains = {}
for p_info in prompt_infos:
    # create a prompt for the chain
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    
    # create and add the chain
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [45]:
print(destinations_str)
print('---------')
print(destination_chains)

physics: Good for answering questions about physics
math: Good for answering math questions
History: Good for answering history questions
computer science: Good for answering computer science questions
---------
{'physics': LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=ChatPromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template="You are a very smart physics professor. You are great at answering questions about physics in a conciseand easy to understand manner. When you don't know the answer to a question you admitthat you don't know.\n\nHere is a question:\n{input}", template_format='f-string', validate_template=True), additional_kwargs={})]), llm=ChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class

We also create a default chain.

In [27]:
# The default chain comes handy when the router doesn't know where to send the input
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

**3. Define the router chain**

In [46]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [52]:
destinations

['physics: Good for answering questions about physics',
 'math: Good for answering math questions',
 'History: Good for answering history questions',
 'computer science: Good for answering computer science questions']

In [54]:
# Create the router template
# Add destinations to the multi-prompt template
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

# Create a router pormpt form the router template
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

# Create the router chain
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# Putting it all together
chain = MultiPromptChain(
    router_chain=router_chain, 
    destination_chains=destination_chains, 
    default_chain=default_chain, verbose=True
)

In [33]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an object that absorbs all radiation that falls on it and emits radiation at all wavelengths. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding the behavior of stars, as well as in the development of technologies such as incandescent light bulbs and infrared cameras."

In [55]:
chain.run("What is DNA?")



> Entering new MultiPromptChain chain...


d:\GitHub\LLMs\LangChain\.env\Lib\site-packages\langchain\chains\llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


None: {'input': 'What is DNA?'}
> Finished chain.


'DNA (Deoxyribonucleic acid) is a molecule that carries genetic information in all living organisms. It is a long, double-stranded helix structure made up of four nucleotide bases: adenine (A), guanine (G), cytosine (C), and thymine (T). The sequence of these bases determines the genetic code that is responsible for the traits and characteristics of an organism. DNA is found in the nucleus of cells and is responsible for the transmission of genetic information from one generation to the next.'

: 